In [1]:
import shutil
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data", one_hot=True)


C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nguye\Anaconda3\envs\env_python3\lib\site-pack

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
X = tf.placeholder("float32", [None, 784]) # 28*28 224*224*3
Y = tf.placeholder("float32", [None, 10])

In [3]:
weights = {
    'wc1': tf.Variable(tf.random_normal([3,3,1, 32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10])),
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([10])), 
}

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def conv2d(x, W, b, stride=1):
    x = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME') 
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, W, b):
    x = tf.reshape(x, shape=[-1, 28, 28, 1]) # None*784 -> None,28, 28, 1
    
    conv1 = conv2d(x, W['wc1'], b['bc1']) # None, 28, 28, 32
    conv1 = maxpool2d(conv1, k=2) # None, 14, 14, 32
    conv2 = conv2d(conv1, W['wc2'], b['bc2']) # None, 14, 14, 64
    conv2 = maxpool2d(conv2, k=2) # None, 7, 7, 64
    
    fc1 = tf.reshape(conv2, [-1, 7*7*64]) # None, 3136
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.add(tf.matmul(fc1,W['wd1']), b['bd1'])
    output = tf.add(tf.matmul(fc1,W['out']), b['out'])
    return output

In [5]:
logits = conv_net(X, weights, biases) # X = 15*784 -> logit = 15*10 -> 

# Compute loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels= Y))
# backwark
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)  
train_op = optimizer.minimize(loss_op)

# Evaluate models 
correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
init = tf.global_variables_initializer()
training_epochs = 1000
log_path = 'conv_logdir2'

tf.summary.scalar("loss_show", loss_op)
tf.summary.scalar("accuracy_show", accuracy)
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    shutil.rmtree(log_path)
    summary_writer = tf.summary.FileWriter(logdir='./'+log_path, graph=tf.get_default_graph())
    for epoch in range(training_epochs):
        # 1 iterator
        #print(sess.run(loss_op))
        image_batch, label_batch = mnist.train.next_batch(10000)
        sess.run(train_op, feed_dict={X: image_batch, Y: label_batch})
        #Display logs per epoch step
        #if (epoch+1) % 10 == 0:
        loss, summary = sess.run([loss_op, merged_summary_op], feed_dict={X: image_batch, Y: label_batch})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(loss))
        summary_writer.add_summary(summary,epoch)
    print("Optimization Finished!")
    print ("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    

summary_writer.close()

Epoch: 0001 cost= 35037.117187500
Epoch: 0002 cost= 27337.251953125
Epoch: 0003 cost= 24974.453125000
Epoch: 0004 cost= 23689.708984375
Epoch: 0005 cost= 20376.296875000
Epoch: 0006 cost= 17569.855468750
Epoch: 0007 cost= 17472.642578125
Epoch: 0008 cost= 17092.277343750
Epoch: 0009 cost= 14382.879882812
Epoch: 0010 cost= 12186.297851562
Epoch: 0011 cost= 11777.262695312
Epoch: 0012 cost= 11800.236328125
Epoch: 0013 cost= 11426.988281250
Epoch: 0014 cost= 10788.674804688
Epoch: 0015 cost= 9856.631835938
Epoch: 0016 cost= 8951.497070312
Epoch: 0017 cost= 8173.166503906
Epoch: 0018 cost= 7724.532226562
Epoch: 0019 cost= 7082.278320312
Epoch: 0020 cost= 6549.316894531
Epoch: 0021 cost= 5853.522949219
Epoch: 0022 cost= 5507.387207031
Epoch: 0023 cost= 5230.899902344
Epoch: 0024 cost= 5317.027832031
Epoch: 0025 cost= 5185.265625000
Epoch: 0026 cost= 4882.837890625
Epoch: 0027 cost= 4572.201660156
Epoch: 0028 cost= 4267.244628906
Epoch: 0029 cost= 3991.407226562
Epoch: 0030 cost= 3841.519287

Epoch: 0254 cost= 346.026306152
Epoch: 0255 cost= 303.597137451
Epoch: 0256 cost= 280.627502441
Epoch: 0257 cost= 315.755645752
Epoch: 0258 cost= 339.749755859
Epoch: 0259 cost= 318.117767334
Epoch: 0260 cost= 271.443450928
Epoch: 0261 cost= 306.465759277
Epoch: 0262 cost= 311.966491699
Epoch: 0263 cost= 307.508392334
Epoch: 0264 cost= 343.288238525
Epoch: 0265 cost= 291.381713867
Epoch: 0266 cost= 309.846160889
Epoch: 0267 cost= 325.946289062
Epoch: 0268 cost= 283.743499756
Epoch: 0269 cost= 312.167785645
Epoch: 0270 cost= 294.126586914
Epoch: 0271 cost= 279.583312988
Epoch: 0272 cost= 299.831573486
Epoch: 0273 cost= 301.279541016
Epoch: 0274 cost= 283.092559814
Epoch: 0275 cost= 300.183227539
Epoch: 0276 cost= 296.772979736
Epoch: 0277 cost= 286.830291748
Epoch: 0278 cost= 281.201049805
Epoch: 0279 cost= 271.971038818
Epoch: 0280 cost= 303.958343506
Epoch: 0281 cost= 272.374237061
Epoch: 0282 cost= 268.709045410
Epoch: 0283 cost= 301.852783203
Epoch: 0284 cost= 283.773712158
Epoch: 0

Epoch: 0511 cost= 131.283752441
Epoch: 0512 cost= 113.205223083
Epoch: 0513 cost= 119.304924011
Epoch: 0514 cost= 124.765190125
Epoch: 0515 cost= 118.203453064
Epoch: 0516 cost= 124.461738586
Epoch: 0517 cost= 144.163024902
Epoch: 0518 cost= 125.574851990
Epoch: 0519 cost= 124.904647827
Epoch: 0520 cost= 139.720779419
Epoch: 0521 cost= 110.258361816
Epoch: 0522 cost= 125.947128296
Epoch: 0523 cost= 122.882690430
Epoch: 0524 cost= 128.925292969
Epoch: 0525 cost= 119.862960815
Epoch: 0526 cost= 101.239265442
Epoch: 0527 cost= 111.248397827
Epoch: 0528 cost= 149.856918335
Epoch: 0529 cost= 112.296165466
Epoch: 0530 cost= 130.976257324
Epoch: 0531 cost= 113.508903503
Epoch: 0532 cost= 108.786872864
Epoch: 0533 cost= 120.756103516
Epoch: 0534 cost= 152.069717407
Epoch: 0535 cost= 130.471984863
Epoch: 0536 cost= 110.463653564
Epoch: 0537 cost= 119.453399658
Epoch: 0538 cost= 111.430839539
Epoch: 0539 cost= 109.235549927
Epoch: 0540 cost= 130.327178955
Epoch: 0541 cost= 98.774665833
Epoch: 05

Epoch: 0773 cost= 51.858955383
Epoch: 0774 cost= 67.132553101
Epoch: 0775 cost= 66.422927856
Epoch: 0776 cost= 62.095912933
Epoch: 0777 cost= 41.832592010
Epoch: 0778 cost= 63.020481110
Epoch: 0779 cost= 60.195075989
Epoch: 0780 cost= 65.610221863
Epoch: 0781 cost= 55.127487183
Epoch: 0782 cost= 53.640426636
Epoch: 0783 cost= 62.709251404
Epoch: 0784 cost= 49.513607025
Epoch: 0785 cost= 63.820968628
Epoch: 0786 cost= 58.986011505
Epoch: 0787 cost= 52.841037750
Epoch: 0788 cost= 58.435382843
Epoch: 0789 cost= 54.675025940
Epoch: 0790 cost= 54.006824493
Epoch: 0791 cost= 53.077362061
Epoch: 0792 cost= 54.323837280
Epoch: 0793 cost= 66.532096863
Epoch: 0794 cost= 55.466125488
Epoch: 0795 cost= 44.721111298
Epoch: 0796 cost= 52.886581421
Epoch: 0797 cost= 57.539630890
Epoch: 0798 cost= 53.771198273
Epoch: 0799 cost= 55.929588318
Epoch: 0800 cost= 56.527938843
Epoch: 0801 cost= 52.090450287
Epoch: 0802 cost= 55.420150757
Epoch: 0803 cost= 44.170082092
Epoch: 0804 cost= 61.019973755
Epoch: 0